# So sánh hiệu quả của các mô hình embedding

In [14]:
import warnings
warnings.filterwarnings("ignore")

In [1]:
from transformers import AutoTokenizer
from transformers import AutoModel

In [3]:
all_MiniLM_L6_v2 = "..\\embedding_models\\all-MiniLM-L6-v2"
distiluse_base_multilingual_cased_v2 = "..\\embedding_models\\distiluse-base-multilingual-cased-v2"
multilingual_e5_small = "..\\embedding_models\\multilingual-e5-small"
sup_SimCSE_VietNamese_phobert_base = "..\\embedding_models\\sup-SimCSE-VietNamese-phobert-base"
vietnamese_bi_encoder = "..\\embedding_models\\vietnamese-bi-encoder"
vietnamese_embedding = "..\\embedding_models\\vietnamese-embedding"

vietnamese_history_dataset = "..\\datasets\\vietnamese_history_dataset\\data.json"

In [ ]:
import pandas as pd
import json
from IPython.display import clear_output

df = pd.read_json(vietnamese_history_dataset)
df.head()

,title,content,type
0,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,chapter
1,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NG...,lesson
2,Những dấu tích của Người tối cổ được tìm thấy ...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nNhững ...,title
3,"Ở giai đoạn đầu, Người tinh khôn sống như thế ...",Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nỞ giai...,title
4,Giai đoạn phát triển của Người tinh khôn có gì...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nGiai đ...,title


In [15]:
df["embedding"] = None
df["embeded"] = None

for idx in range(len(df)):
    df["embedding"][idx] = df["title"][idx] + ": " + df["content"][idx]

df.head()

,title,content,type,embedding,embeded
0,BUỔI ĐẦU LỊCH SỬ NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA,chapter,BUỔI ĐẦU LỊCH SỬ NƯỚC TA: Chương: BUỔI ĐẦU LỊC...,None
1,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA,Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NG...,lesson,THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA: Chương: BUỔ...,None
2,Những dấu tích của Người tối cổ được tìm thấy ...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nNhững ...,title,Những dấu tích của Người tối cổ được tìm thấy ...,None
3,"Ở giai đoạn đầu, Người tinh khôn sống như thế ...",Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nỞ giai...,title,"Ở giai đoạn đầu, Người tinh khôn sống như thế ...",None
4,Giai đoạn phát triển của Người tinh khôn có gì...,Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nGiai đ...,title,Giai đoạn phát triển của Người tinh khôn có gì...,None


In [21]:
import torch

max_seq_length = 2048
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)
if device.type == "cuda":
    convert_to_tensor = True
else:
    convert_to_tensor = False
convert_to_numpy = not convert_to_tensor
batch_size = 32
output_value = "sentence_embedding"
normalize_embeddings = False # Chuẩn hóa L2

cpu


| Loại Embedding       | Mô tả ngắn gọn                                                                                  |
| -------------------- | ----------------------------------------------------------------------------------------------- |
| `sentence_embedding` | Một vector đại diện cho **toàn bộ câu**. Thường là đầu ra của `[CLS]` hoặc trung bình các token |
| `token_embedding`    | Một list các vector – **mỗi token (từ, subword)** trong câu có một vector riêng                 |


| Tiêu chí                     | `sentence_embedding`                         | `token_embedding`                              |
| ---------------------------- | -------------------------------------------- | ---------------------------------------------- |
| 📌 Đầu ra                    | 1 vector có kích thước cố định (e.g., 768-d) | 1 list các vector (số lượng = số token)        |
| 📏 Độ dài                    | Không phụ thuộc độ dài câu                   | Phụ thuộc vào số lượng token của câu           |
| 🧮 Ứng dụng chính            | Semantic search, clustering, classification  | NER, POS tagging, highlighting, attention mask |
| 🧪 Dễ sử dụng                | Rất dễ – chỉ cần dùng như vector             | Khó hơn – cần xử lý token alignment            |
| ⚡ Tốc độ tính toán           | Nhanh hơn, ít tài nguyên hơn                 | Chậm hơn, tốn RAM/GPU hơn                      |
| 🔍 Mức độ chi tiết ngữ nghĩa | Tổng quan toàn câu                           | Chi tiết từng phần trong câu                   |


Dùng sentence_embedding khi:

- Semantic search: tìm tài liệu phù hợp với câu hỏi

- Text classification: phân loại câu, bài báo, bình luận

- Clustering: gom nhóm văn bản theo ý nghĩa

- Tính cosine similarity giữa các câu

→ Ưu điểm: Gọn, nhanh, dễ dùng


Dùng token_embedding khi:

- Named Entity Recognition (NER)

- Highlight cụm từ quan trọng (từng token)

- Cần sự chi tiết đến từng phần nhỏ của câu (subword level)

- Làm việc với attention, visualization layer

→ Ưu điểm: Cực kỳ linh hoạt và chi tiết, nhưng cũng phức tạp hơn


# distiluse-base-multilingual-cased-v2

In [37]:
from sentence_transformers import SentenceTransformer

embedding_model_path = sup_SimCSE_VietNamese_phobert_base
model = SentenceTransformer(embedding_model_path)
model.max_seq_length = max_seq_length

No sentence-transformers model found with name ..\embedding_models\sup-SimCSE-VietNamese-phobert-base. Creating a new one with mean pooling.


In [41]:
sentences = [sentence for sentence in df["embedding"]]
print(sentences)

['BUỔI ĐẦU LỊCH SỬ NƯỚC TA: Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA', 'THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA: Chương: BUỔI ĐẦU LỊCH SỬ NƯỚC TA\nBài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA', 'Những dấu tích của Người tối cổ được tìm thấy ở đâu?: Bài: THỜI NGUYÊN THUỶ TRÊN ĐẤT NƯỚC TA\nNhững dấu tích của Người tối cổ được tìm thấy ở đâu?: Thời xa xưa, nước ta là một vùng rừng núi rậm rạp với nhiều hang động, mái đá, nhiều sông suối, có vùng ven biển dài; khí hậu hai mùa nóng - lạnh rõ rệt, thuận lợi cho cuộc sống của cỏ cây, muông thú và con người. Vào những năm 1960 - 1965, các nhà khảo cổ học đã lần lượt phát hiện được hàng loạt di tích của Người tối cổ. Ở các hang Thẩm Khuyên, Thẩm Hai (Lạng Sơn), trong lớp đất chứa nhiều than, xương vật cổ cách đây 40 - 30 vạn năm, người ta phát hiện được những chiếc răng của Người tối cổ. Ở một số nơi khác như núi Đọ, Quan Yên (Thanh Hoá), Xuân Lộc (Đồng Nai), người ta phát hiện được nhiều công cụ đá thô sơ dùng để chặt, đập; nhiều mảnh đá ghè mỏng. Ở nhiều chỗ.', 

In [ ]:
for idx in range(len(df)):
    sentences = df["embedding"][idx]
    df["embeded"][idx] = model.encode(
        sentences=sentences,
        batch_size=batch_size,
        device=device,
        show_progress_bar=False,
        output_value=output_value,
        convert_to_numpy=convert_to_numpy,
        convert_to_tensor=convert_to_tensor,
        normalize_embeddings=normalize_embeddings,
    )
    
    clear_output(wait=True)
    progress = (idx + 1) / len(df) * 100
    print(f"Progress: {progress:.2f}%")

df.head()

# multilingual-e5-small

# sup-SimCSE-VietNamese-phobert-base

# vietnamese-bi-encoder

# vietnamese-embedding